<h1>
<center>
Module 4: Tree Building assignment
</center>
</h1>
<div class=h1_cell>

You will be working with the loan table again.

</div>

In [32]:
import pandas as pd

with open('/Users/Noah/Documents/My Documents/2018:2019/2. Winter/CIS 399/Assignment4/loan_wrangled_week3.csv', 'r') as f:
    loan_table = pd.read_csv(f)
    
with open('/Users/Noah/Documents/My Documents/2018:2019/2. Winter/CIS 399/Assignment4/loan_results_week2.csv', 'r') as f:
    loan_results = pd.read_csv(f)


In [57]:
from library_w19_week4 import *
%who function

accuracy	 build_pred	 build_tree_iter	 compute_prediction	 f1	 find_best_splitter	 generate_table	 gig	 gini	 
informedness	 predictor_case	 probabilities	 tree_predictor	 


In [34]:
pd.set_option('display.max_columns', None)
loan_table.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,lam_bin,lam_Low,lam_Average,lam_High,ch_bad,ch_good,ch_nan,apin_bin,apin_low,apin_average,apin_high,apin_nan
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0,0,128.000000,1,0,0,0,Low,1,0,0,0,1,0,low,1,0,0,0
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1,0,66.000000,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1,0,120.000000,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1,0,141.000000,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0


In [35]:
loan_table.columns.values

array(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Loan_Status', 'no_lam', 'filled_lam', 'pa_Rural', 'pa_Semiurban',
       'pa_Urban', 'pa_nan', 'lam_bin', 'lam_Low', 'lam_Average',
       'lam_High', 'ch_bad', 'ch_good', 'ch_nan', 'apin_bin', 'apin_low',
       'apin_average', 'apin_high', 'apin_nan'], dtype=object)

<hr>
<h1>
1: OHE Dependents
</h1>
If you take a look at the Dependents column below, you will see that it is viewed as a categorical column with 4 unique values. So we should be able to use get_dummies on it.

In [36]:
loan_table.describe(include='all')

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,lam_bin,lam_Low,lam_Average,lam_High,ch_bad,ch_good,ch_nan,apin_bin,apin_low,apin_average,apin_high,apin_nan
count,601,611,599,614,582,614.000000,614.000000,592.000000,600.00000,564.000000,614,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.0,614,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614,614.000000,614.000000,614.000000,614.0
unique,2,2,4,2,2,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN
top,Male,Yes,0,Graduate,No,NaN,NaN,NaN,NaN,NaN,Semiurban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low,NaN,NaN,NaN,NaN,NaN,NaN,low,NaN,NaN,NaN,NaN
freq,489,398,345,480,500,NaN,NaN,NaN,NaN,NaN,233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,556,NaN,NaN,NaN,NaN,NaN,NaN,607,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,5403.459283,1621.245798,146.412162,342.00000,0.842199,NaN,0.687296,0.035831,146.412162,0.291531,0.379479,0.328990,0.0,NaN,0.905537,0.074919,0.019544,0.144951,0.773616,0.081433,NaN,0.988599,0.008143,0.003257,0.0
std,NaN,NaN,NaN,NaN,NaN,6109.041673,2926.248369,85.587325,65.12041,0.364878,NaN,0.463973,0.186019,84.037468,0.454838,0.485653,0.470229,0.0,NaN,0.292710,0.263475,0.138540,0.352339,0.418832,0.273722,NaN,0.106250,0.089945,0.057026,0.0
min,NaN,NaN,NaN,NaN,NaN,150.000000,0.000000,9.000000,12.00000,0.000000,NaN,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.0
25%,NaN,NaN,NaN,NaN,NaN,2877.500000,0.000000,100.000000,360.00000,1.000000,NaN,0.000000,0.000000,100.250000,0.000000,0.000000,0.000000,0.0,NaN,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,NaN,1.000000,0.000000,0.000000,0.0
50%,NaN,NaN,NaN,NaN,NaN,3812.500000,1188.500000,128.000000,360.00000,1.000000,NaN,1.000000,0.000000,129.000000,0.000000,0.000000,0.000000,0.0,NaN,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,NaN,1.000000,0.000000,0.000000,0.0
75%,NaN,NaN,NaN,NaN,NaN,5795.000000,2297.250000,168.000000,360.00000,1.000000,NaN,1.000000,0.000000,164.750000,1.000000,1.000000,1.000000,0.0,NaN,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,NaN,1.000000,0.000000,0.000000,0.0


Go ahead and ohe the `Dependents` column - use `dep` as prefix. And look at above to see the count. It has empties, right? So set the appropriate flag.

In [44]:
#ohe code here

one_hot_dep = pd.get_dummies(loan_table["Dependents"], prefix="dep", dummy_na = True)
loan_table = loan_table.join(one_hot_dep)

ValueError: columns overlap but no suffix specified: Index(['dep_0', 'dep_1', 'dep_2', 'dep_3+', 'dep_nan'], dtype='object')

In [45]:
loan_table.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,lam_bin,lam_Low,lam_Average,lam_High,ch_bad,ch_good,ch_nan,apin_bin,apin_low,apin_average,apin_high,apin_nan,dep_0,dep_1,dep_2,dep_3+,dep_nan
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0,0


<hr>
<h1>
2: Find the best depth
</h1>
<p>
<div class=h1_cell>
<p>
Explore depths from 1 to 5. Build a tree for each depth and test it. Plot the results.
<p>
First, define the columns to use. I do that for you below.
</div>

In [58]:
column_candidates = [
        #Dependents
        'dep_0', 'dep_1', 'dep_2', 'dep_3+',
        #ApplicantIncome
       'apin_low', 'apin_high', 'apin_average',
        #Property_Area
        'pa_Rural', 'pa_Semiurban','pa_Urban',
        #LoanAmount
        'lam_Low', 'lam_Average', 'lam_High',
        #Credit_History
        'ch_bad', 'ch_good']

In [59]:
tree1 = build_tree_iter(loan_table, column_candidates, 'Loan_Status', {'max-depth': 1}) # should get 2 paths
print(len(tree1['paths']))
tree1['paths']

2


[{'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 0,
  'gig_score': 0.12559965113188631},
 {'conjunction': [('ch_bad_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.12559965113188631}]

<div class=h1_cell>
<p>
As reminder, a tree with only one splitter node is called a stump. You can see that `ch_bad` is the best splitter with only one node. Thought question: As we build deeper trees below, is it possible for the root node to be something other than `ch_bad`?
</div>

In [60]:
tree2 = build_tree_iter(loan_table, column_candidates, 'Loan_Status', {'max-depth': 2}) # should get 4 paths
print(len(tree2['paths']))
tree2['paths']

4


[{'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('apin_average_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.019292789018833467},
 {'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('apin_average_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 0,
  'gig_score': 0.019292789018833467},
 {'conjunction': [('ch_bad_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('pa_Semiurban_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.006462619030620731},
 {'conjunction': [('ch_bad_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('pa_Semiurban_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.006462619030620731}]

In [61]:
tree3 = build_tree_iter(loan_table, column_candidates, 'Loan_Status', {'max-depth': 3}) # should get 7 paths
print(len(tree3['paths']))
tree3['paths']

7


[{'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('apin_average_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.019292789018833467},
 {'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('apin_average_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('pa_Urban_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 0,
  'gig_score': 0.0014037192421204325},
 {'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('apin_average_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('pa_Urban_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 0,
  'gig_score': 0.0014037192421204325},
 {'conjunction': [('ch_bad_0',
    <function library_w19_week4.build_pred.<lo

In [62]:
tree4 = build_tree_iter(loan_table, column_candidates, 'Loan_Status', {'max-depth': 4}) # should get 13 paths
print(len(tree4['paths']))
tree4['paths']

13


[{'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('apin_average_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.019292789018833467},
 {'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('apin_average_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('pa_Urban_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('dep_3+_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 0,
  'gig_score': 0.01942421089143239},
 {'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('apin_average_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('pa_Urban_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('dep_3+_0',
    <function library_w19_week4.build

In [63]:
tree5 = build_tree_iter(loan_table, column_candidates, 'Loan_Status', {'max-depth': 5}) # should get 20 paths
print(len(tree5['paths']))
#tree5['paths']

20


<div class=h1_cell>
  <h1>You have your 5 trees</h1>
<p>
Add them to your results table.
</div>

In [77]:
loan_results['tree_1'] = loan_table.apply(lambda row: tree_predictor(row, tree1), axis=1)
loan_results['tree_1_type'] = loan_results.apply(lambda row: predictor_case(row, pred="tree_1", target="Loan_Status"), axis=1)
loan_results.head(1)

,Loan_Status,pred_1,pred_1_type,pred_2,pred_2_type,tree_1,tree_1_type
0,1,1,true_positive,1,true_positive,1,true_positive


In [82]:
loan_results['tree_2'] = loan_table.apply(lambda row: tree_predictor(row, tree2), axis=1)
loan_results['tree_2_type'] = loan_results.apply(lambda row: predictor_case(row, pred="tree_2", target="Loan_Status"), axis=1)
loan_results.head(1)

,Loan_Status,pred_1,pred_1_type,pred_2,pred_2_type,tree_1,tree_1_type,tree_3,tree_3_type,tree_4,tree_4_type,tree_5,tree_5_type,tree_2,tree_2_type
0,1,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive


In [83]:
loan_results['tree_3'] = loan_table.apply(lambda row: tree_predictor(row, tree3), axis=1)
loan_results['tree_3_type'] = loan_results.apply(lambda row: predictor_case(row, pred="tree_3", target="Loan_Status"), axis=1)
loan_results.head(1)

,Loan_Status,pred_1,pred_1_type,pred_2,pred_2_type,tree_1,tree_1_type,tree_3,tree_3_type,tree_4,tree_4_type,tree_5,tree_5_type,tree_2,tree_2_type
0,1,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive


In [84]:
loan_results['tree_4'] = loan_table.apply(lambda row: tree_predictor(row, tree4), axis=1)
loan_results['tree_4_type'] = loan_results.apply(lambda row: predictor_case(row, pred="tree_4", target="Loan_Status"), axis=1)
loan_results.head(1)

,Loan_Status,pred_1,pred_1_type,pred_2,pred_2_type,tree_1,tree_1_type,tree_3,tree_3_type,tree_4,tree_4_type,tree_5,tree_5_type,tree_2,tree_2_type
0,1,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive


In [85]:
loan_results['tree_5'] = loan_table.apply(lambda row: tree_predictor(row, tree5), axis=1)
loan_results['tree_5_type'] = loan_results.apply(lambda row: predictor_case(row, pred="tree_5", target="Loan_Status"), axis=1)
loan_results.head(1)

,Loan_Status,pred_1,pred_1_type,pred_2,pred_2_type,tree_1,tree_1_type,tree_3,tree_3_type,tree_4,tree_4_type,tree_5,tree_5_type,tree_2,tree_2_type
0,1,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive


Get the value counts on each tree.

In [86]:
tree_1_types = loan_results['tree_1_type'].value_counts()
tree_2_types = loan_results['tree_2_type'].value_counts()
tree_3_types = loan_results['tree_3_type'].value_counts()
tree_4_types = loan_results['tree_4_type'].value_counts()
tree_5_types = loan_results['tree_5_type'].value_counts()

ok, ready to check out our measures.

In [87]:
print((accuracy(tree_1_types), f1(tree_1_types), informedness(tree_1_types)))

(0.8094462540716613, 0.8764519535374868, 0.4104956556082149)


In [88]:
print((accuracy(tree_2_types), f1(tree_2_types), informedness(tree_2_types)))

(0.8110749185667753, 0.8776371308016878, 0.41286532385466024)


In [89]:
print((accuracy(tree_3_types), f1(tree_3_types), informedness(tree_3_types)))

(0.8110749185667753, 0.8776371308016878, 0.41286532385466024)


In [90]:
print((accuracy(tree_4_types), f1(tree_4_types), informedness(tree_4_types)))

(0.8127035830618893, 0.8785638859556495, 0.4180736571879937)


In [91]:
print((accuracy(tree_5_types), f1(tree_5_types), informedness(tree_5_types)))

(0.8192182410423453, 0.8802588996763754, 0.46161631121642976)


<div class=h1_cell>
<p>
Should we stop at 5? It looks like our measures are continuing to go up. I'll let you stop here but in practice it would be good to find where depth is optimal.
  <p>
Let's try at least one change to the gig cutoff. Use `0.01` and level 5.
</div>

In [126]:
tree6 = build_tree_iter(loan_table, column_candidates, 'Loan_Status', {'max-depth':5, 'gig-cutoff': .01})
tree6['paths']
print(len(tree6['paths']))

3


Wow. We went from 20 paths to 3 by using gig-cutoff. See how tree6 does.

In [97]:
loan_results['tree_6'] = loan_table.apply(lambda row: tree_predictor(row, tree6), axis=1)
loan_results['tree_6_type'] = loan_results.apply(lambda row: predictor_case(row, pred="tree_6", target="Loan_Status"), axis=1)
loan_results.head(1)

,Loan_Status,pred_1,pred_1_type,pred_2,pred_2_type,tree_1,tree_1_type,tree_3,tree_3_type,tree_4,tree_4_type,tree_5,tree_5_type,tree_2,tree_2_type,tree_6,tree_6_type
0,1,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive


In [98]:
tree_6_types = loan_results['tree_6_type'].value_counts()
print((accuracy(tree_6_types), f1(tree_6_types), informedness(tree_6_types)))


(0.8110749185667753, 0.8776371308016878, 0.41286532385466024)


Same as tree3 but with 4 fewer paths: 3 instead of 7.

<hr>
<h1>
3: Order the paths in a tree according to most used
</h1>
<p>
<div class=h1_cell>
<p>
One of the cool things about building our own code for doing predictions with decision trees is that we can explore various options directly. Let's do that now. Look at this code in `tree_predictor`:
<p>
```
for path in tree['paths']:
```
<p>
It is doing a sequential and ordered enumeration of the paths in a tree. We can gain some efficiency by ordering the paths, right? If we can determine which paths are most likely to succeed, we can try those paths first. In practice, this means reordeing the list of dictionaries in tree['paths'] so most used path is first, and least used path is last. I'd like you to write some code that will reorder `tree['paths']` on this basis of most used.
<p>
I'll break it up into pieces. First, define a function `path_id` that accepts a row and a tree and returns the index of the path that succeeded. So if the first path in tree['paths'] succeeded, you would return its index 0.
</div>

In [106]:
def path_id(row, tree):
  
    assert (len(tree['paths']) > 0), "Tree must have at least one path"
    i = 0
    for path in tree["paths"]:
        conjuncts = path['conjunction']
        result = map(lambda tuple: tuple[1](row), conjuncts)
        if all(result):
            return i
        i+=1

<div class=h1_cell>

Now apply the function against the loan table using tree1 and produce a series called `pcounts1`. Remember that tree1 has only 2 paths.
</div>

In [108]:
pcounts1 = loan_table.apply(lambda row: path_id(row, tree1), axis=1)

pcounts1.value_counts()

1    525
0     89
dtype: int64

<div class=h1_cell>
Looks like path 1 (i.e., `tree1['paths'][1]`) is the big winner.
  <p>
    Let's see what tree2 looks like. It has 4 paths.
</div>

In [109]:
pcounts2 = loan_table.apply(lambda row: path_id(row, tree2), axis=1)

pcounts2.value_counts()

3    322
2    203
1     88
0      1
dtype: int64

<div class=h1_cell>
  Looks like path 0 should definitely be tried last. In fact, it looks like the list is in exactly reverse order in terms of path usage.
  <p>
Let's look at tree3.
</div>

In [110]:
pcounts3 = loan_table.apply(lambda row: path_id(row, tree3), axis=1)

pcounts3.value_counts()

6    267
4    187
2     57
5     55
1     31
3     16
0      1
dtype: int64

<div class=h1_cell>

Let's use `tree3` (i.e., `pcounts3`) as our focus. Turn `pcounts3.value_counts()` into a dictionary.
</div>

In [117]:
pdict3 = dict(pcounts3.value_counts())
pdict3

{6: 267, 4: 187, 2: 57, 5: 55, 1: 31, 3: 16, 0: 1}

<div class=h1_cell>

Now turn the dictionary into a list of tuples. Be careful. Generators can bite you.
</div>

In [118]:
plist3 = list(pdict3.items())
plist3

[(6, 267), (4, 187), (2, 57), (5, 55), (1, 31), (3, 16), (0, 1)]

<div class=h1_cell>
This is kind of odd. It appears that converting to a list of tuples has also sorted the list by value in reverse order. I cannot find any documentation that says this should happen. So I am not taking chances. I am going to go ahead and sort `plist3` in a separate function. You shoud too unless you can point me to documentation that says pulling tuples from a dictionary does sorting.
</div>

In [144]:
from operator import itemgetter #from StackOverflow
plist3 = list(reversed(sorted(plist3,key=itemgetter(1))))
#plist3 = list(reversed(sorted(plist3))) #returns exactly what it was, used just to be safe
plist3

[(6, 267), (4, 187), (2, 57), (5, 55), (1, 31), (3, 16), (0, 1)]

<div class=h1_cell>
Finally, build a new path list using plist3 as a guide.
</div>

In [147]:
new_paths = []
for path in plist3:
    #print(path[0])
    new_paths.append(tree3["paths"][path[0]])
new_paths

[{'conjunction': [('ch_bad_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('pa_Semiurban_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('dep_2_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.0021858669417437193},
 {'conjunction': [('ch_bad_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('pa_Semiurban_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('lam_Average_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.0052148575352743975},
 {'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('apin_average_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('pa_Urban_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 0,
  'gig_score': 0.00140371924

I can now reset `tree3["paths"]` to `new_paths`. Cool.

<hr>
<h2>
Put it all together in a function
</h2>
<p>
<div class=h1_cell>
<p>
Please write `reorder_paths` to produce a list of paths that is sorted on most frequently used. You can use the output of the function as:
<p>
<pre>
    ordered_paths3 = reorder_paths(loan_table, tree3)
    tree3['paths'] = ordered_paths3
</pre>
<p>
I'll give you some examples below that you can test on - match my output. Note that I am printing some debugging info from inside my function. I am printing the sorted list I come up with. You do not have to print this but it is useful for debugging.
</div>

In [159]:
from operator import itemgetter

def reorder_paths(table, tree):
    pcounts = loan_table.apply(lambda row: path_id(row, tree), axis=1)
    pdict = dict(pcounts.value_counts())
    plist = list(pdict.items())
    plist = list(reversed(sorted(plist,key=itemgetter(1))))
    print(plist)
    new_paths = []
    for path in plist:
        new_paths.append(tree["paths"][path[0]])
    return new_paths

In [160]:
tree1['paths']  #before we reorder

[{'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 0,
  'gig_score': 0.12559965113188631},
 {'conjunction': [('ch_bad_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.12559965113188631}]

In [161]:
reorder_paths(loan_table, tree1)  # First line printed is just for debugging

[(1, 525), (0, 89)]


[{'conjunction': [('ch_bad_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.12559965113188631},
 {'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 0,
  'gig_score': 0.12559965113188631}]

In [162]:
tree2['paths']  #before we reorder

[{'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('apin_average_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.019292789018833467},
 {'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('apin_average_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 0,
  'gig_score': 0.019292789018833467},
 {'conjunction': [('ch_bad_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('pa_Semiurban_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.006462619030620731},
 {'conjunction': [('ch_bad_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('pa_Semiurban_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.006462619030620731}]

In [163]:
reorder_paths(loan_table, tree2)

[(3, 322), (2, 203), (1, 88), (0, 1)]


[{'conjunction': [('ch_bad_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('pa_Semiurban_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.006462619030620731},
 {'conjunction': [('ch_bad_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('pa_Semiurban_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.006462619030620731},
 {'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('apin_average_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 0,
  'gig_score': 0.019292789018833467},
 {'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('apin_average_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.019292789018833467}]

In [164]:
reorder_paths(loan_table, tree3)

[(6, 267), (4, 187), (2, 57), (5, 55), (1, 31), (3, 16), (0, 1)]


[{'conjunction': [('ch_bad_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('pa_Semiurban_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('dep_2_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.0021858669417437193},
 {'conjunction': [('ch_bad_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('pa_Semiurban_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('lam_Average_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 1,
  'gig_score': 0.0052148575352743975},
 {'conjunction': [('ch_bad_1',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('apin_average_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>),
   ('pa_Urban_0',
    <function library_w19_week4.build_pred.<locals>.<lambda>(row)>)],
  'prediction': 0,
  'gig_score': 0.00140371924

<hr>
<h1>Write it out</h1>
<div class=h1_cell>

We changed both loan_table and loan results. Write them out.
</div>

In [165]:
with open('/Users/Noah/Documents/My Documents/2018:2019/2. Winter/CIS 399/Assignment4/loan_table_week4.csv', 'w') as f:
  loan_table.to_csv(f, encoding='utf-8', index=False)
  
with open('/Users/Noah/Documents/My Documents/2018:2019/2. Winter/CIS 399/Assignment4/loan_results_week4.csv', 'w') as f:
  loan_results.to_csv(f, encoding='utf-8', index=False)

<hr>
<h1>Don't forget library functions</h1>
<div class=h1_cell>

Add them to `library_w19_week4b.py`. You defined `library_w19_week4.py` in intro module. You are now defining a new version of it with functions from this notebook. I added the `b` to keep them clear.
</div>